In [ ]:
## path to datasets
datadir='../Datasets/'

In [ ]:
import igraph as ig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

# Power grid graph

We start with a test of robustness of an empirical graph that does not exhibit power law.

Network robustness is an important practical topic for power grids.

First we load data

In [ ]:
gr = ig.Graph.Read_Ncol(datadir+'GridEurope/gridkit_europe-highvoltage.edges', directed=False)

In [ ]:
gr = gr.simplify()

In [ ]:
gr.ecount()

In [ ]:
X = pd.read_csv(datadir+'GridEurope/gridkit_europe-highvoltage.vertices')

In [ ]:
X

In [ ]:
gr = ig.Graph.Read_Ncol(datadir+'GridEurope/gridkit_europe-highvoltage.edges', directed=False)
gr = gr.simplify()
## read the vertices along with some attributes
X = pd.read_csv(datadir+'GridEurope/gridkit_europe-highvoltage.vertices')
idx = [int(i) for i in gr.vs['name']]
sorterIndex = dict(zip(idx,range(len(idx))))
X['Rank'] = X['v_id'].map(sorterIndex)
X.sort_values(['Rank'], ascending=[True],inplace=True)
X.dropna(inplace=True)
gr.vs['longitude'] = list(X['lon'])
gr.vs['latitude'] = list(X['lat'])
gr.vs['type'] = list(X['typ'])
gr.vs['layout'] = [(v['longitude'],v['latitude']) for v in gr.vs()]
gr.vs['size'] = 3
gr.es['color'] = 'grey'
gr.vs['color'] = 'black'

In [ ]:
ig.plot(gr, layout = ig.Layout(gr.vs['layout']))

We want to concentrate on an Iberic peninsula. Note that we select a giant component of the graph selected by latitude and longitude as e.g. nodes located on islands are not part of the giant component of this subgraph.

In [ ]:
V = [v for v in gr.vs() if v['latitude']>36 and v['latitude']<44 and v['longitude']>-10 and v['longitude']<4]

In [ ]:
gr_spain = gr.subgraph(V)

In [ ]:
ig.plot(gr_spain, layout = ig.Layout(gr_spain.vs['layout']))

In [ ]:
## subgraph of Grid -- giant component on Iberic peninsula
V = [v for v in gr.vs() if v['latitude']>36 and v['latitude']<44 and v['longitude']>-10 and v['longitude']<4]
gr_spain = gr.subgraph(V).clusters().giant()
ly = ig.Layout(gr_spain.vs['layout'])
ly.mirror(1)
ig.plot(gr_spain, 'grid_iberic_giant.eps', layout=ly, bbox=(0,0,300,300))

We check the node count and average node degree 

In [ ]:
gr_spain.vcount()

In [ ]:
np.mean(gr_spain.degree())

This function given a graph `gr` tests removing nodes from this graph until `stop_count` nodes are left.
* If `fun` is `"random"` then we assume random failure model.
* If `fun` is `"degree"` then we assume targetted attack, in which sequentially a node with the highest degree in the remaining network is removed.
* If `fun` is `"between"` then we assume targetted attack, in which sequentially a node with the highest betweenness centrality in the remaining network is removed.

The function returns order parameter sequence (i.e. the fraction of nodes in the giant component of the remaining graph) and the final graph.

In [ ]:
def single_run(gr, fun, stop_count=1):
    ref_vcount = gr.vcount()
    gr = gr.copy()
    order = [1.0]
    while gr.vcount() > stop_count:
        if fun == "random":
            to_delete = random.randint(0, gr.vcount() - 1)
        elif fun == "degree":
            m = max(gr.degree())
            am = [i for i, j in enumerate(gr.degree()) if j == m]
            to_delete = random.choice(am)
        elif fun == "between":
            b = gr.betweenness()
            m = max(b)
            am = [i for i, j in enumerate(b) if j == m]
            to_delete = random.choice(am)
        else:
            raise Exception("unknown value of parameter fun")
        gr.delete_vertices(to_delete)
        order.append(gr.clusters().giant().vcount() / ref_vcount)
    return order, gr

We run all variants of failure scenarios:

In [ ]:
res_rnd = single_run(gr_spain, "random")[0]

In [ ]:
res_rnd

In [ ]:
res_degree = single_run(gr_spain, "degree")[0]

In [ ]:
res_between = single_run(gr_spain, "between")[0]

In [ ]:
gr_spain.vcount()

Observe that order parameter:
1. Falls slowest when `fun` is `"random"`.
2. For the considered graph `"betweenness"` attack is more effective (note that this graph does not follow power law in the degree distriubtion and is embedded on a plane, with edges connecting nodes that are close geographically).
3. When almost all nodes are removed order parameter increases, but this is a case that is not very interesting (we are mostly interested in the left parts of the plots)

In [ ]:
plt.plot(100 * np.arange(600) / gr_spain.vcount(), res_rnd[:600], ":", color="black")
plt.plot(100 * np.arange(600) / gr_spain.vcount(), res_degree[:600], "--", color="black")
plt.plot(100 * np.arange(600) / gr_spain.vcount(), res_between[:600], "-", color="black")
plt.plot(100 * np.arange(600) / gr_spain.vcount(), 1 - np.arange(600) / gr_spain.vcount(), color="gray")
plt.xlabel("% of removed nodes")
plt.ylabel("order parameter")
plt.legend(["random", "degree", "betweenness", "% of nodes"], loc=1);
#plt.savefig('op_power_grid.eps', format='eps')

We test a single scenario under maximum betweenness attack when 5% of nodes are removed. Colors on the plot represent different components of the graph.

In [ ]:
_, gr_sel = single_run(gr_spain, "between", gr_spain.vcount()*0.95)

In [ ]:
ly = ig.Layout(gr_sel.vs['layout'])
ly.mirror(1)
ig.plot(gr_sel.clusters(), 'grid_iberic_5pct_attack.eps', layout=ly)#, bbox=(0,0,300,300))

In [ ]:
len(gr_sel.clusters())

Check the number of components

In [ ]:
gr_sel.clusters().sizes()

And distribution of their sizes. We can see that we have many small components.

In [ ]:
plt.hist(gr_sel.clusters().sizes(), 50);

In [ ]:
df = pd.DataFrame({'size': gr_sel.clusters().sizes()})

In [ ]:
df['size'].value_counts().sort_index()

Let us now consider maximum degree attack.

In [ ]:
_, gr_sel = single_run(gr_spain, "degree", gr_spain.vcount()*0.95)

In [ ]:
plt.hist(gr_sel.clusters().sizes(), 50);

In [ ]:
df = pd.DataFrame({'size': gr_sel.clusters().sizes()})
df['size'].value_counts().sort_index()

This time we get more very small components components, but the largest components are bigger.

# Watts-Strogatz

In this scenario we consider a Watts-Strogratz graph with 2000 nodes and 4000 edges. We vary $p$ parameter from $0.00$ to $1.00$.

In [ ]:
g000 = ig.Graph.Watts_Strogatz(1, 2000, 2, 0.00)

In [ ]:
g001 = ig.Graph.Watts_Strogatz(1, 2000, 2, 0.01)

In [ ]:
g005 = ig.Graph.Watts_Strogatz(1, 2000, 2, 0.05)

In [ ]:
g100 = ig.Graph.Watts_Strogatz(1, 2000, 2, 1.00)

In [ ]:
plt.plot(100 * np.arange(1000) / g000.vcount(), single_run(g000, "random")[0][:1000], ":", color="black")
plt.plot(100 * np.arange(1000) / g001.vcount(), single_run(g001, "random")[0][:1000], "--", color="black")
plt.plot(100 * np.arange(1000) / g005.vcount(), single_run(g005, "random")[0][:1000], "--", color="gray")
plt.plot(100 * np.arange(1000) / g100.vcount(), single_run(g100, "random")[0][:1000], "-", color="black")
plt.plot(100 * np.arange(1000) / g000.vcount(), 1 - np.arange(1000) / g000.vcount(), color="gray")
plt.xlabel("% of removed nodes")
plt.ylabel("order parameter")
plt.legend(["p=0%", "p=1%", "p=5%", "p=100%", "% of nodes"], loc=1);
#plt.savefig('ws_random.eps', format='eps')

Observe that for random removal of nodes even a small fraction of rewired edges makes the graph much more robust.

In [ ]:
plt.plot(100 * np.arange(1000) / g000.vcount(), single_run(g000, "degree")[0][:1000], ":", color="black")
plt.plot(100 * np.arange(1000) / g001.vcount(), single_run(g001, "degree")[0][:1000], "--", color="black")
plt.plot(100 * np.arange(1000) / g005.vcount(), single_run(g005, "degree")[0][:1000], "--", color="gray")
plt.plot(100 * np.arange(1000) / g100.vcount(), single_run(g100, "degree")[0][:1000], "-", color="black")
plt.plot(100 * np.arange(1000) / g000.vcount(), 1 - np.arange(1000) / g000.vcount(), color="gray")
plt.xlabel("% of removed nodes")
plt.ylabel("order parameter")
plt.legend(["p=0%", "p=1%", "p=5%", "p=100%", "% of nodes"], loc=1);
#plt.savefig('ws_degree.eps', format='eps')

For Watts-Strogatz graph we see that the effectiveness of maximum degree attack does not show a simple pattern. We see that fro a regular graph ($p=0.00$) this type of attack is not very effective. Then as the $p$ insreases the relationship is non-monotonous.

In [ ]:
plt.plot(100 * np.arange(1000) / g000.vcount(), single_run(g000, "between")[0][:1000], ":", color="black")
plt.plot(100 * np.arange(1000) / g001.vcount(), single_run(g001, "between")[0][:1000], "--", color="black")
plt.plot(100 * np.arange(1000) / g005.vcount(), single_run(g005, "between")[0][:1000], "--", color="gray")
plt.plot(100 * np.arange(1000) / g100.vcount(), single_run(g100, "between")[0][:1000], "-", color="black")
plt.plot(100 * np.arange(1000) / g000.vcount(), 1 - np.arange(1000) / g000.vcount(), color="gray")
plt.xlabel("% of removed nodes")
plt.ylabel("order parameter")
plt.legend(["p=0%", "p=1%", "p=5%", "p=100%", "% of nodes"], loc=1);
#plt.savefig('ws_between.eps', format='eps')

Note that the betweenness scenario is the most expensive to calculate.

We note that this kind of attack is very effective for Watts-Strogatz graph, expecially when $p$ is very small.

# Power law

Here we investigate the scenario where we simulate a graph whose degree distribution follows the power law.

In all compared scenarios we generate a graph with 2000 nodes and mean degree 4 and change the power law exponent only. It takes values from the set $\{2, 2.5, 3, 4\}$.

In [ ]:
def gen_pl(exponent):
    g = ig.Graph.Static_Power_Law(2000, 4000, exponent)
    return (single_run(g, "random")[0], single_run(g, "degree")[0], single_run(g, "between")[0])

Note that computations are slow, which is mostly caused by the fact that  betweenness centrality is expensive to calculate.

In [ ]:
r4, d4, b4 = gen_pl(4)
r3, d3, b3 = gen_pl(3)
r25, d25, b25 = gen_pl(2.5)
r2, d2, b2 = gen_pl(2)

In [ ]:
plt.plot(100 * np.arange(1000) / 2000, r4[:1000], ":", color="black")
plt.plot(100 * np.arange(1000) / 2000, r3[:1000], "--", color="black")
plt.plot(100 * np.arange(1000) / 2000, r25[:1000], "--", color="gray")
plt.plot(100 * np.arange(1000) / 2000, r2[:1000], "-", color="black")
plt.plot(100 * np.arange(1000) / 2000, 1 - np.arange(1000) / 2000, color="gray")
plt.xlabel("% of removed nodes")
plt.ylabel("order parameter")
plt.legend(["exponent=4", "exponent=3", "exponent=2.5", "exponent=2", "% of nodes"], loc=1);
#plt.savefig('pl_random.eps', format='eps')

In [ ]:
plt.plot(100 * np.arange(1000) / 2000, d4[:1000], ":", color="black")
plt.plot(100 * np.arange(1000) / 2000, d3[:1000], "--", color="black")
plt.plot(100 * np.arange(1000) / 2000, d25[:1000], "--", color="gray")
plt.plot(100 * np.arange(1000) / 2000, d2[:1000], "-", color="black")
plt.plot(100 * np.arange(1000) / 2000, 1 - np.arange(1000) / 2000, color="gray")
plt.xlabel("% of removed nodes")
plt.ylabel("order parameter")
plt.legend(["exponent=4", "exponent=3", "exponent=2.5", "exponent=2", "% of nodes"], loc=1);
#plt.savefig('pl_degree.eps', format='eps')

In [ ]:
plt.plot(100 * np.arange(1000) / 2000, b4[:1000], ":", color="black")
plt.plot(100 * np.arange(1000) / 2000, b3[:1000], "--", color="black")
plt.plot(100 * np.arange(1000) / 2000, b25[:1000], "--", color="gray")
plt.plot(100 * np.arange(1000) / 2000, b2[:1000], "-", color="black")
plt.plot(100 * np.arange(1000) / 2000, 1 - np.arange(1000) / 2000, color="gray")
plt.xlabel("% of removed nodes")
plt.ylabel("order parameter")
plt.legend(["exponent=4", "exponent=3", "exponent=2.5", "exponent=2", "% of nodes"], loc=1);
#plt.savefig('pl_between.eps', format='eps')

Note that for all strategies the lower the value of the power law exponent the faster the order parameter decreases (even if we correct for the fact that for the value of the exponent equal to $2$ the initial graph has the higher number of isolated nodes).

# Assortative

Next we turn our analysis to checking how network assortativity influences the order parameter.

We use an adaptation of Xulvi-Brunet and Sokolov algorithm, but we start from a graph on 2000 nodes and 4000 edges with degree distribution power law exponent equal to 3.

In [ ]:
## Xulvi-Brunet and Sokolov algorithm with power law underlying graph
def XBS(q, assortative):
    g = ig.Graph.Static_Power_Law(2000, 4000, 3)
    g.es['touched'] = False
    ec = g.ecount()
    while True:
        re = np.random.choice(ec, 2, replace=False)
        nodes = list(g.es[re[0]].tuple+g.es[re[1]].tuple)
        if len(set(nodes))==4:
            ## with proba q, wire w.r.t. assortativity, else randomly
            if np.random.random()<q:
                idx = np.argsort(g.degree(nodes))
                if assortative:
                    e1 = (nodes[idx[0]],nodes[idx[1]])
                    e2 = (nodes[idx[2]],nodes[idx[3]])
                else:
                    e1 = (nodes[idx[0]],nodes[idx[3]])
                    e2 = (nodes[idx[1]],nodes[idx[2]])
            else:
                np.random.shuffle(nodes)
                e1 = (nodes[0],nodes[1])
                e2 = (nodes[2],nodes[3])
            if g.get_eid(e1[0], e1[1], directed=False, error=False)+\
               g.get_eid(e2[0], e2[1], directed=False, error=False) == -2:
                    g.delete_edges(re)
                    g.add_edge(e1[0],e1[1],touched=True)
                    g.add_edge(e2[0],e2[1],touched=True)
            else:
                g.es[re[0]]['touched']=True
                g.es[re[1]]['touched']=True
        if sum(g.es['touched']) == g.ecount():
            break
    return (single_run(g, "random")[0], single_run(g, "degree")[0], single_run(g, "between")[0])

We compare positive, negative, and no modification in assortativity of the original power law graph scenarios.

In [ ]:
ra, da, ba = XBS(3/4, True)
rd, dd, bd = XBS(3/4, False)
r0, d0, b0 = XBS(0, True)

In [ ]:
plt.plot(100 * np.arange(1000) / 2000, ra[:1000], ":", color="black")
plt.plot(100 * np.arange(1000) / 2000, rd[:1000], "--", color="black")
plt.plot(100 * np.arange(1000) / 2000, r0[:1000], "-", color="black")
plt.plot(100 * np.arange(1000) / 2000, 1 - np.arange(1000) / 2000, color="gray")
plt.xlabel("% of removed nodes")
plt.ylabel("order parameter")
plt.legend(["assortative", "dissortative", "neutral", "% of nodes"], loc=1);
#plt.savefig('ass_random.eps', format='eps')

In [ ]:
plt.plot(100 * np.arange(1000) / 2000, da[:1000], ":", color="black")
plt.plot(100 * np.arange(1000) / 2000, dd[:1000], "--", color="black")
plt.plot(100 * np.arange(1000) / 2000, d0[:1000], "-", color="black")
plt.plot(100 * np.arange(1000) / 2000, 1 - np.arange(1000) / 2000, color="gray")
plt.xlabel("% of removed nodes")
plt.ylabel("order parameter")
plt.legend(["assortative", "dissortative", "neutral", "% of nodes"], loc=1);
#plt.savefig('ass_degree.eps', format='eps')

In [ ]:
plt.plot(100 * np.arange(1000) / 2000, ba[:1000], ":", color="black")
plt.plot(100 * np.arange(1000) / 2000, bd[:1000], "--", color="black")
plt.plot(100 * np.arange(1000) / 2000, b0[:1000], "-", color="black")
plt.plot(100 * np.arange(1000) / 2000, 1 - np.arange(1000) / 2000, color="gray")
plt.xlabel("% of removed nodes")
plt.ylabel("order parameter")
plt.legend(["assortative", "dissortative", "neutral", "% of nodes"], loc=1);
#plt.savefig('ass_between.eps', format='eps')

We can observe that higher assortitivity makes the graph more prone to random failures but less sensitive to both maximum degree and maximum betweenness attacks (both of them have approximately similar effectiveness for this graph, though max betweenness attack is slightly more effective).

# Communities

The last experiment we perform is with a graph that contains communities. We used ABCD graph generator with $\xi\in\{0.0001,0.01,0.1,1.0\}$.

Again we tune the graphs to have 2000 nodes and average degree approximately equal to 4.

The remaining parameters are described in Datasets/ABCD/readme_edge.txt file.

In [ ]:
def read_graph(name):
    g = ig.Graph.Read_Ncol(datadir+'ABCD/edge'+name+'.dat',directed=False)
    print("node count: ", g.vcount())
    print("mean degree:", np.mean(g.degree()))
    return single_run(g, "random")[0], single_run(g, "degree")[0], single_run(g, "between")[0]

In [ ]:
r00001, d00001, b00001 = read_graph('00001')
r001, d001, b001 = read_graph('001')
r01, d01, b01 = read_graph('01')
r1, d1, b1 = read_graph('1')

In [ ]:
plt.plot(100 * np.arange(1000) / 2000, r00001[:1000], ":", color="black")
plt.plot(100 * np.arange(1000) / 2000, r001[:1000], "--", color="black")
plt.plot(100 * np.arange(1000) / 2000, r01[:1000], "--", color="gray")
plt.plot(100 * np.arange(1000) / 2000, r1[:1000], "-", color="black")
plt.plot(100 * np.arange(1000) / 2000, 1 - np.arange(1000) / 2000, color="gray")
plt.xlabel("% of removed nodes")
plt.ylabel("order parameter")
plt.legend(["xi=0.0001", "x=0.01", "xi=0.1", "xi=1.0", "% of nodes"], loc=1);
#plt.savefig('xi_random.eps', format='eps')

In [ ]:
plt.plot(100 * np.arange(1000) / 2000, d00001[:1000], ":", color="black")
plt.plot(100 * np.arange(1000) / 2000, d001[:1000], "--", color="black")
plt.plot(100 * np.arange(1000) / 2000, d01[:1000], "--", color="gray")
plt.plot(100 * np.arange(1000) / 2000, d1[:1000], "-", color="black")
plt.plot(100 * np.arange(1000) / 2000, 1 - np.arange(1000) / 2000, color="gray")
plt.xlabel("% of removed nodes")
plt.ylabel("order parameter")
plt.legend(["xi=0.0001", "x=0.01", "xi=0.1", "xi=1.0", "% of nodes"], loc=1);
#plt.savefig('xi_degree.eps', format='eps')

In [ ]:
plt.plot(100 * np.arange(1000) / 2000, b00001[:1000], ":", color="black")
plt.plot(100 * np.arange(1000) / 2000, b001[:1000], "--", color="black")
plt.plot(100 * np.arange(1000) / 2000, b01[:1000], "--", color="gray")
plt.plot(100 * np.arange(1000) / 2000, b1[:1000], "-", color="black")
plt.plot(100 * np.arange(1000) / 2000, 1 - np.arange(1000) / 2000, color="gray")
plt.xlabel("% of removed nodes")
plt.ylabel("order parameter")
plt.legend(["xi=0.0001", "x=0.01", "xi=0.1", "xi=1.0", "% of nodes"], loc=1);
#plt.savefig('xi_between.eps', format='eps')

We can see that again - max degree and max betweenness attacks are much more effective than random node removal. As in earlier scenarios we note that max betweenness attack is more efficient than max degree attack. In particular note that this is especially visible for low values of $\xi$. For values of $\xi$ in range $[0.1,1.0]$ we do not observe significant variability (actually for the max degree attack they are identical for the generated graphs).